In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import os
import numpy as np
from keras.utils import np_utils
from keras.preprocessing import sequence

from sklearn.metrics import accuracy_score
from Protein_Dataset import Protein_Dataset

Using TensorFlow backend.


# Data preparation

Load train and test data

In [2]:
ds = Protein_Dataset("train/", "test/")

NameError: name 'os' is not defined

Data utils

In [14]:

def build_prediction(part_predictions, original_length, stride):
    max_len = part_predictions.shape[1]
    
    final_prediction = []
    
    if len(part_predictions) != 1:
        final_prediction = np.concatenate(part_predictions[:-1], axis=0)

    el_left = original_length % max_len
    start_element = stride
    
    if el_left + stride > max_len:
        start_element = max_len - el_left

    final_part_prediction = part_predictions[-1][start_element:start_element + el_left]

    final_prediction = np.concatenate([final_prediction, final_part_prediction], axis=0)
    
    return final_prediction


In [15]:

def predict(x, session, method, maxlen=400, stride=100):
    prepared = []; idx = []; original_len = []; model_len = []
    
    curr_idx = 0
    
    for seq in x:
        original_len.append(len(seq))
        
        splited = split_seq(seq, maxlen, stride)
        
        model_len.extend(list(map(len, splited)))
        splited = sequence.pad_sequences(splited, maxlen, value=-1, padding="post")
        
        prepared.extend(splited)
        idx.append([curr_idx, curr_idx+len(splited)])
        curr_idx += len(splited)
    
    predictions = session.run(method, feed_dict={
        x_raw: np.array(prepared),
        seq_len: np.array(model_len)
    })
    
    final_predictions = []
    for i, seq_idx in enumerate(idx):
        seq_predictions = predictions[slice(seq_idx[0], seq_idx[1])]

        final_predictions.append(build_prediction(seq_predictions, original_len[i], stride))
        
    return np.array(list(map(lambda x:make_pred(x, len(x), class_table), final_predictions)))


In [16]:
def score(x, y, session, method, maxlen=400, stride=100):
    y_pred = predict(x, session, method)
    y_pred = list(map(lambda x: list(map(lambda z: class_table.index(z) + 1, x)), y_pred))
    return np.mean([accuracy_score(y[i], y_pred[i]) for i in range(len(y))])

In [17]:
def acc_score(y_true, y_pred):
    y_pred = list(map(lambda x: list(map(lambda z: class_table.index(z) + 1, x)), y_pred))
    return np.mean([accuracy_score(y_true[i], y_pred[i]) for i in range(len(y_true))])

# Model

In [21]:
input_class = len(acid_table)
output_class = len(class_table) + 1 # plus one for padding

learning_rate = 0.01
seq_max_len = 400
n_units = 64

In [22]:
x_raw = tf.placeholder(tf.int32, [None, seq_max_len], name="x_raw")
y_raw = tf.placeholder(tf.int32, [None, seq_max_len], name="y_raw")
seq_len = tf.placeholder(tf.int32, [None], name="seq_len")

x = tf.one_hot(x_raw, input_class, dtype=tf.float32, name="one_hot")

In [23]:
def lstm_cell(reuse=tf.get_variable_scope().reuse):
    cell = tf.contrib.rnn.LSTMCell(n_units, reuse=reuse)
    return tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.8)

In [24]:
fw_cell = lstm_cell(); bw_cell = lstm_cell()

output, state = tf.nn.bidirectional_dynamic_rnn(fw_cell, bw_cell, x, seq_len, dtype=tf.float32)

In [25]:
con_out = tf.concat(output, 2)

In [26]:
flatten = tf.reshape(con_out, [-1, 2 * n_units * seq_max_len])

In [27]:
h1_num_units = int(np.mean([2 * n_units * seq_max_len, output_class * seq_max_len]))

In [28]:
weights = {
    "h1"  : tf.Variable(tf.truncated_normal(shape=[2 * n_units * seq_max_len, h1_num_units], stddev=0.1)),
    "out" : tf.Variable(tf.truncated_normal(shape=[h1_num_units, output_class * seq_max_len], stddev=0.1))
}
bias = {
    "h1"  : tf.Variable(tf.truncated_normal(shape=[h1_num_units])),
    "out" : tf.Variable(tf.truncated_normal(shape=[output_class * seq_max_len]))
}

In [29]:
h1 = tf.nn.relu(tf.matmul(flatten, weights["h1"]) + bias["h1"])
flat_logits = tf.matmul(h1, weights["out"]) + bias["out"]

In [30]:
logits = tf.reshape(flat_logits, shape=[-1, seq_max_len, output_class])

In [31]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_raw, logits=logits)

In [32]:
mask = tf.to_float(tf.not_equal(y_raw, 0))

In [33]:
norm_loss = loss * mask

In [37]:
l2_reg = tf.nn.l2_loss(weights['h1'])

In [38]:
beta = 0.01

In [39]:
mean_loss = tf.reduce_mean(tf.reduce_sum(norm_loss) / tf.to_float(seq_len) + l2_reg * beta)

In [40]:
pred = tf.arg_max(tf.nn.softmax(logits), 2)

In [41]:
train_step = tf.train.AdamOptimizer(learning_rate).minimize(mean_loss)

In [42]:
num_epochs = 10
batch_size = 50
disp_batch = 10

In [1]:
1+1

2

In [ ]:
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    
    
    for e in range(num_epochs):
        avg_loss = 0.; num_iter = 0
        for x_batch, y_batch, len_batch in dynamic_iter(train, batch_size, maxlen=seq_max_len):
            
            _, c = sess.run([train_step, mean_loss], feed_dict={
                x_raw: x_batch,
                y_raw: y_batch,
                seq_len: len_batch
            })
            
            num_iter += 1; avg_loss += c
            
            if num_iter % disp_batch == 0:
                
                test_pred = predict(test[:, 0], sess, pred)
                
                print("Batch {}".format(num_iter))
                print("Example: ")
                
                dem_idx = np.random.randint(0, len(test))
                
                print("Real:      " + make_pred(test[dem_idx, 1], len(test[dem_idx, 1]), class_table))
                
                print("Predicted: " + test_pred[dem_idx])
                
                score = acc_score(test[:, 1], test_pred)
                
                print("Accuracy: " + str(score))
                
        print("Epoch {} done! Average loss: ".format(str(e)) + str(avg_loss / num_iter))

In [83]:
j_net_pred = []
for name in test_names:
    j_net_pred.append("".join(read_file(os.path.join("JNet_data/", name+".jnet"))[1].strip("\njnetpred:").split(",")))

acc_score(test[:, 1], j_net_pred)

0.82065223269391485